In [1]:
from ai_toolkits.llms.pydantic_provider.models import create_openai_like

In [ ]:
from ai_toolkits.llms.pydantic_provider.models import create_openai_like

from openai import AsyncClient
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import ModelSettings
import nest_asyncio
nest_asyncio.apply()


model = create_openai_like(
    model_name = "Qwen3-32B-AWQ",
    api_key="2e7a7e90-0dd1-4cad-ac43-9a6ee86cebdc",
    base_url="https://vesta.yxqiche.com/vesta_test/v2/engine_online/flow_schedule/schedule_run_openai/v1",
    default_headers={
        "AccountId": "Algorithm",
        "AccountToken": "3fc524bd5e1a6f7fe8c9480b74cc2bc3",
        "AppId": "1964929122673102848",
        "TenantId": "YIXIN",
        "TokenKey": "v8znd4bj64awm90pk410l679ya5rj8ev",
        "RequestId": "UAT_0987654"
    },
    extra_body={
        "chat_template_kwargs": {
            "enable_thinking": True,
            "separate_reasoning": False
        }
    }
)


agent = Agent(model)

res = agent.run_sync("写一个python函数，计算两个数的和")